In [1]:
# --- SECTION 1 ---
# Libraries and data loading
from copy import deepcopy
from sklearn.datasets import load_diabetes
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import numpy as np

diabetes = load_diabetes()

train_size = 400
train_x, train_y = diabetes.data[:train_size], diabetes.target[:train_size]
test_x, test_y = diabetes.data[train_size:], diabetes.target[train_size:]

np.random.seed(123456)

In [2]:
# --- SECTION 2 ---
# Create the ensemble

# Define the ensemble's size, learning rate and decision tree depth
ensemble_size = 50
learning_rate = 0.1
base_classifier = DecisionTreeRegressor(max_depth=3)

# Create placeholders for the base learners and each step's prediction
base_learners = []
# Note that the initial prediction is the target variable's mean
previous_predictions = np.zeros(len(train_y)) + np.mean(train_y)

# Create the base learners
for _ in range(ensemble_size):
    # Start by calcualting the pseudo-residuals
    errors = train_y - previous_predictions
    
    # Make a deep copy of the base classifier and train it on the
    # pseudo-residuals
    learner = deepcopy(base_classifier)
    learner.fit(train_x, errors)
    
    # Predict the residuals on the train set
    predictions = learner.predict(train_x)
    
    # Multiply the predictions with the learning rate and add the results
    # to the previous prediction
    previous_predictions = previous_predictions + learning_rate*predictions
    
    # Save the base learner
    base_learners.append(learner)

In [3]:
# --- SECTION 3 ---
# Evaluate the ensemble

# Start with the train set's mean
previous_predictions = np.zeros(len(test_y)) + np.mean(train_y)

# For each base learner predict the pseudo-residuals for the test set and
# add them to the previous prediction, after multiplying with the learning rate
for learner in base_learners:
    predictions = learner.predict(test_x)
    previous_predictions = previous_predictions + learning_rate*predictions

In [4]:
# --- SECTION 4 ---
# Print the metrics
r2 = metrics.r2_score(test_y, previous_predictions)
mse = metrics.mean_squared_error(test_y, previous_predictions)

print('Gradient Boosting:')
print('R-squared: %.2f' % r2)
print('MSE: %.2f' % mse)

Gradient Boosting:
R-squared: 0.59
MSE: 2253.34
